# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

## **CANDIDATE**: Himanshu Sharma
## **TASK**: Build GenAI-Based Features assessment

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
# %pip install openai httpx==0.27.2
%pip install faiss-cpu langchain langchain-community tiktoken gspread gspread_dataframe --quiet

In [ ]:
# Import libraries
import pandas as pd
import faiss
import numpy as np
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import re

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# ----------------------------- Google Sheets Integration Setup -----------------------------

# Import the Google Colab authentication module. This is required to access Google APIs (like Sheets or Drive) securely.
from google.colab import auth

# Launches an authentication flow in the Colab environment.
# The user will be prompted to sign in with their Google account and authorize access.
auth.authenticate_user()

# Import gspread – a Python API client for Google Sheets.
# It allows programmatic access to read, write, and manage Google Sheets.
import gspread

# Import the default credentials mechanism from Google's OAuth2 client.
# This will retrieve the current user's authenticated credentials (after `authenticate_user()`).
from google.auth import default

# Import helper to write Pandas DataFrames directly to Google Sheets
from gspread_dataframe import set_with_dataframe

# Retrieve the default credentials and project info (we only need creds here)
creds, _ = default()

# Create an authorized gspread client using the authenticated credentials.
# This object (gc) will be used to create and update Google Sheets programmatically.
gc = gspread.authorize(creds)


In [ ]:
# ----------------------------- Step 1: Read Product and Email Data from Google Sheets -----------------------------

# Define a reusable function to read a specific Google Sheet tab as a DataFrame
# The function converts the Google Sheet tab into a downloadable CSV format via Google's Visualization API
def read_data_frame(document_id, sheet_name):
    url = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(url)  # Read the CSV into a Pandas DataFrame

# Google Sheet document ID containing both 'products' and 'emails' sheets
document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'

# Load the product catalog into a DataFrame
# This contains fields like product ID, name, description, stock amount, etc.
products_df = read_data_frame(document_id, 'products')

# Load the list of customer emails into a DataFrame
# Each row contains the email ID, subject, and body content
emails_df = read_data_frame(document_id, 'emails')


# ----------------------------- Step 2: Setup OpenAI Client for GPT-4o Access -----------------------------

# Create an OpenAI client instance to interact with GPT-4o via a custom hosted API endpoint
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',  # Custom proxy base for OpenAI API
    api_key='a0BIj000002c315MAA'  # Temporary API key provided for this coding assessment
)

# Task 1. Classify emails

In [ ]:
# ----------------------------- Step 3: Classify Emails Using GPT-4o -----------------------------

# Define a function that uses GPT-4o to classify an email as either a 'product inquiry' or 'order request'
def classify_email(subject, body):
    # Construct a clear, contextual prompt to instruct the model to classify the email intent
    # The model is instructed to respond with only one label to avoid ambiguity
    prompt = f"""You are an intelligent email assistant working for a critical business.
Classify this email as either 'product inquiry' or 'order request'.

Subject: {subject}
Body: {body}

Return only one of the two: product inquiry OR order request."""

    # Send the prompt to GPT-4o via the chat completion endpoint
    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract and normalize the response (e.g., remove whitespace and convert to lowercase)
    return result.choices[0].message.content.strip().lower()

# Initialize a list to store classification results
email_classification = []

# Iterate over each email in the 'emails_df' DataFrame
# For each email, call the classify_email function and append the result to a structured list
for _, row in emails_df.iterrows():
    category = classify_email(row['subject'], row['message'])  # row['message'] contains the body of the email
    email_classification.append({'email_id': row['email_id'], 'category': category})  # Store result with email ID

# Convert the classification results into a new DataFrame
# This will later be written to the 'email-classification' output sheet
email_classification_df = pd.DataFrame(email_classification)

##The code below is specifically designed to handle a full product catalog containing over 100,000 items without exceeding token limits. It uses a Retrieval-Augmented Generation (RAG) approach powered by FAISS, which ensures that only the most relevant product information is retrieved and passed to the language model, rather than including the entire catalog in the prompt. This makes the solution scalable, efficient, and well-suited for real-world e-commerce scenarios.


In [ ]:
# ----------------------------- Step 4: Build FAISS Index for Product Inquiry Responses -----------------------------

# Construct product documents for the vector database
# Each row from the product catalog is converted into a semantic document using LangChain's `Document` class
# This document includes a structured natural language description of each product
product_docs = [
    Document(
        page_content=(
            f"Product Name: {row['name']}, "
            f"Category: {row['category']}, "
            f"Description: {row['description']}, "
            f"Season: {row['seasons']}, "
            f"Stock: {row['stock']}, "
            f"Price: {row['price']}"
        ),
        metadata={"product ID": row["product_id"]}
    )
    for _, row in products_df.iterrows()
]

# Initialize an OpenAI embedding model to convert product descriptions into dense vector embeddings
# This step translates text into high-dimensional numeric vectors that represent semantic meaning
embedding_model = OpenAIEmbeddings(
    openai_api_key='a0BIj000002c315MAA',
    openai_api_base="https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/"  # Custom API proxy for GPT-4o
)

# Use FAISS to build a fast vector store from the embedded product documents
# FAISS enables similarity search, allowing us to retrieve the top-K most relevant products for a query
vectorstore = FAISS.from_documents(product_docs, embedding_model)

# Convert FAISS index into a retriever object
# The retriever will fetch top 5 semantically similar products for any product inquiry
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Create a Retrieval-Augmented Generation (RAG) QA chain using LangChain
# This integrates GPT-4o with the retriever so that only relevant product information is passed to the LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        openai_api_key='a0BIj000002c315MAA',
        model="gpt-4o",
        openai_api_base="https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/"
    ),
    retriever=retriever
)


In [ ]:
# ----------------------------- FAISS-based Product Name Matching -----------------------------

def match_product_by_name(product_name_query):
    """
    Uses FAISS vector store to find the best matching product from the catalog
    based on a free-text name or description returned by the LLM.

    This function is crucial when the LLM output does not exactly match the product
    names in the catalog. Instead of relying on strict string matching, we leverage
    semantic search using embeddings to handle variations, typos, and fuzzy phrasing.
    """

    # Perform a similarity search on the FAISS vector index using the LLM's output as the query.
    # The result is a ranked list of product documents (wrapped as LangChain Document objects),
    # ordered by how semantically close they are to the given input.
    docs = vectorstore.similarity_search(product_name_query, k=1)  # Get top-1 match

    # In rare cases where no similar product is found, we return None gracefully.
    if not docs:
        return None

    # Extract the metadata field 'product ID' from the best matching document.
    # This ID was attached to the FAISS index when we constructed it earlier from the product catalog.
    matched_product_id = docs[0].metadata['product ID']

    # Retrieve and return the full product row from the original product DataFrame using the matched ID.
    # This allows the rest of the pipeline (e.g., stock checking, fulfillment) to work with structured product data.
    return products_df[products_df['product_id'] == matched_product_id].iloc[0]


# Task 2. Process order requests

In [ ]:
# ----------------------------- Step 5: Extract and Fulfill Orders (Supports Partial Fulfillment) -----------------------------

# Initialize lists to store structured output for order statuses and email responses
order_status_records = []     # Will hold product-level order processing results
order_responses = []          # Will hold AI-generated customer email replies

# Create a modifiable dictionary of stock levels indexed by product_id
# This will be updated in real time as orders are processed
updated_stock = products_df.set_index("product_id")['stock'].to_dict()

# ----------------------------- Helper Function: Extract Order Info from Email Using GPT-4o -----------------------------

def extract_order_info(email_text):
    """
    Uses GPT-4o to parse unstructured order emails and extract product names and quantities.
    A special rule is included: if the customer says they want 'all' stock, GPT will return -999999.
    """
    prompt = f"""You are a professional customer support agent working for a critical business.
Extract product name and quantity from this order request email.

Email: {email_text}

Return as a list in format: product_name ### quantity
(e.g., T-shirt ### 2). The quantity should always be a number. If the customer gives a range like 3 to 4, or 3-4, return quantity as lower limit i.e. 3.
If a customer wants all available stock, return quantity as '-999999'."""

    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip().split("\n")

# ----------------------------- Main Loop: Process Each Email -----------------------------

for _, row in emails_df.iterrows():
    email_id, subject, body = row["email_id"], row["subject"], row["message"]

    # Retrieve the classification category ('order request' or 'product inquiry')
    category = email_classification_df.loc[
        email_classification_df['email_id'] == email_id,
        'category'
    ].values[0]

    # Process only order request emails
    if category == "order request":
        order_lines = extract_order_info(body)  # Use GPT to extract order lines
        order_summary = ""  # To accumulate response text for the customer

        for line in order_lines:
            if '###' not in line:
                continue  # Skip lines not matching expected format

            # Split line into product name and quantity
            product_name, quantity = line.split('###')
            # Remove any non-numeral characters from quantity using regular expressions
            quantity = re.sub(r"[^0-9-]", "", quantity.strip())

            # Check if quantity is empty after removing non-numeral characters
            if quantity:
                quantity = int(quantity)
            else:
                # Handle the case where quantity is empty (e.g., set to 0 or skip)
                # You might want to log this scenario or provide a default value
                quantity = 0  # Example: Setting quantity to 0

            product_name = product_name.strip()

            # Match product by FAISS
            try:
                matched_product = match_product_by_name(product_name)
            except:
                continue

            product_id = matched_product['product_id']
            stock = updated_stock.get(product_id, 0)

            # Determine how much can be fulfilled
            if quantity == -999999 or stock >= quantity:
                # Case: all stock requested or enough stock available
                status = "created"
                fulfilled_qty = stock if quantity == -999999 else quantity
                updated_stock[product_id] -= fulfilled_qty
            elif stock > 0:
                # Case: only partial stock available
                status = "partially fulfilled"
                fulfilled_qty = stock
                updated_stock[product_id] = 0
            else:
                # Case: no stock available
                status = "out of stock"
                fulfilled_qty = 0

            # Record the outcome of this order line
            order_status_records.append({
                "email ID": email_id,
                "product ID": product_id,
                "quantity": quantity,
                "status": status
            })

            # Add line to order summary for email generation
            order_summary += (
                f"{product_name} - Requested: {quantity}, "
                f"Fulfilled: {fulfilled_qty}, Status: {status}\n"
            )

        # ----------------------------- Generate Customer-Facing Email Using GPT-4o -----------------------------

        response_prompt = f"""You are a professional customer support agent working for a critical business.
Below is the order summary for a customer:

{order_summary}

Write a polite and professional email informing them of the order status.
If any item is partially or not fulfilled, explain why and offer alternatives or suggest restocking options."""

        reply = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": response_prompt}]
        )

        # Store the generated response tied to the email ID
        order_responses.append({
            "email_id": email_id,
            "response": reply.choices[0].message.content.strip()
        })

# Task 3. Handle product inquiry

In [ ]:
# ----------------------------- Step 6: Respond to Product Inquiries using RAG -----------------------------

# Initialize a list to store AI-generated responses to product inquiry emails
inquiry_responses = []

# Loop through each email in the dataset
for _, row in emails_df.iterrows():
    email_id = row['email_id']  # Unique identifier for the email

    # Look up the classification of the current email (e.g., 'product inquiry' or 'order request')
    category = email_classification_df.loc[
        email_classification_df['email_id'] == email_id,
        'category'
    ].values[0]

    # Only process emails that have been classified as product inquiries
    if category == "product inquiry":
        # Use the Retrieval-Augmented Generation (RAG) pipeline to answer the question
        # The qa_chain retrieves the top 5 relevant products using FAISS, then feeds them into GPT-4o
        result = qa_chain.run(row['message'])

        # Store the response along with the email ID in a structured format
        inquiry_responses.append({
            "email_id": email_id,
            "response": result.strip()
        })

# Task 4. Write All Outputs to Google Spreadsheet

In [ ]:
# ----------------------------- Step 7: Write All Outputs to Google Spreadsheet -----------------------------

# Create a new Google Spreadsheet document named as per the assessment requirement
# This will serve as the final submission artifact containing all categorized outputs
# Check if the spreadsheet already exists
try:
    output_document = gc.open('Solving Business Problems with AI - Output___')
    print("Spreadsheet found. Updating existing spreadsheet.")
except gspread.exceptions.SpreadsheetNotFound:
    # Create a new Google Spreadsheet document named as per the assessment requirement
    # This will serve as the final submission artifact containing all categorized outputs
    output_document = gc.create('Solving Business Problems with AI - Output')
    print("Spreadsheet not found. Creating a new spreadsheet.")

# ----------------------------- Sheet 1: Email Classification -----------------------------

# Add a new sheet for email classification results
sheet1 = output_document.add_worksheet(title="email-classification", rows=100, cols=2)

# Set the header for the classification sheet
sheet1.update([['email ID', 'category']], 'A1:B1')

# Write the classification DataFrame to the sheet
set_with_dataframe(sheet1, email_classification_df)

# ----------------------------- Sheet 2: Order Status -----------------------------

# Convert recorded order line statuses into a DataFrame
order_status_df = pd.DataFrame(order_status_records)

# Add a new sheet for order processing status
sheet2 = output_document.add_worksheet(title="order-status", rows=100, cols=4)

# Set the header for the order status sheet
sheet2.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Write the order status DataFrame to the sheet
set_with_dataframe(sheet2, order_status_df)

# ----------------------------- Sheet 3: Order Responses -----------------------------

# Convert order email responses into a DataFrame
order_response_df = pd.DataFrame(order_responses)

# Add a sheet for GPT-4o generated responses to order requests
sheet3 = output_document.add_worksheet(title="order-response", rows=100, cols=2)

# Set the header for the order response sheet
sheet3.update([['email ID', 'response']], 'A1:B1')

# Write the order response DataFrame to the sheet
set_with_dataframe(sheet3, order_response_df)

# ----------------------------- Sheet 4: Inquiry Responses -----------------------------

# Convert inquiry email responses into a DataFrame
inquiry_response_df = pd.DataFrame(inquiry_responses)

# Add a sheet for GPT-4o generated responses to product inquiries
sheet4 = output_document.add_worksheet(title="inquiry-response", rows=100, cols=2)

# Set the header for the inquiry response sheet
sheet4.update([['email ID', 'response']], 'A1:B1')

# Write the inquiry response DataFrame to the sheet
set_with_dataframe(sheet4, inquiry_response_df)

# ----------------------------- Make Spreadsheet Publicly Accessible -----------------------------

# Share the Google Spreadsheet with anyone who has the link (read-only access)
output_document.share('', perm_type='anyone', role='reader')

# Print the shareable link to access the completed output document
print(f"\n✅ All tasks completed successfully. The Google sheet can be accessed here:\nhttps://docs.google.com/spreadsheets/d/{output_document.id}")

Spreadsheet not found. Creating a new spreadsheet.

✅ All tasks completed successfully. The Google sheet can be accessed here:
https://docs.google.com/spreadsheets/d/1TxVsgksB2NGfRygQxBnaQVQrgVzDd9pu-grwtCEI_bU
